# Recommendation Systems

We have seen how Recommender/Recommendation Systems have played an integral parts in the success of Amazon (Books, Items), Pandora/Spotify (Music), Google (News, Search), YouTube (Videos), etc.  For Amazon, these systems bring more than 30% of their total revenues. For Netflix service, 75% of movies that people watch are based on some sort of recommendation.

> The goal of Recommendation Systems is to find what is likely to be of interest to the user. This enables organizations to offer a high level of personalization and customer-tailored services.

## Three Main Types

- non-personalized
- content-based
- collaborative filtering

### Non-Personalized Recommendations

![screenshot of youtube's homepage](images/youtube-nonpersonalizedrecommendations.png)

YouTube is notorious for putting non-personalized content on their homepage (although they tailor recommendations in other places)

These recommendations are based purely on the popularity of the item!

#### Advantages
- Super easy (computationally and for the user to understand)
- Items are usually popular for a reason
- No cold-start issue

#### Disadvantages
- Not personalized
- New items won’t gain traction

## Content-Based

![screenshot found online of someone's 'made for you' recommendations from spotify](images/spotify-contentrecommendations.png)

[Image Source](https://www.howtogeek.com/393291/already-a-spotify-fan-here-are-6-new-features-you-might-have-missed/)

Content-based recommendations are based on the properties/attributes of the items, where the items you've rated highly (or, in Spotify's case, listened to recently or often) are then compared against the properties/attributes of other items, and those items are then recommended if they're considered 'similar'.

What items are 'similar'? Depends on your similarity metric:

![similarity metrics comparison](images/similaritymetrics.png)

[Image Source: "What Similarity Metric Should You Use for Your Recommendation System?](https://medium.com/bag-of-words/what-similarity-metric-should-you-use-for-your-recommendation-system-b45eb7e6ebd0) <- useful reading!

Those are just 3 examples, there are others (Jaccard index, Euclidian similarity) - but the point is you take some mathematical understanding of the items and find which ones are 'nearby' in some sense.

#### Advantages:
- Easy and transparent
- No cold start issue
- Recommend items to users with unique tastes

#### Disadvantages:
- Requires some type of tagging of items
- Overspecialization to certain types of items

## Collaborative Filtering

![collaborative filtering utility matrix example](images/collaborativefiltering.png)

[Image Source](https://www.incubegroup.com/blog/recommender-system-for-private-banking/)

Use both User and Item data! Use past behavior of many users (how they've rated many items) to find similarities either between users or between items (either user-based or item-based) to recommend new things.

We build a Utility/Rating Matrix to capture many users' ratings of many different items - a matrix that, in practice, tends to be quite _sparse_ (see all the blanks in just this tiny example above).

Then, for _some_ models, we use **_MATH_** (namely, matrix factorization) to fill in those blanks, based upon similar users' ratings of similar items.

More specifically, it finds factor matrices which result in the ratings it has - decomposing the actual Utility Matrix into component pieces that explain it. These component pieces, matrices themselves, can be thought of as 'latent' or 'inherent' features of the items and users! The math then comes in, as we calculate the dot products in order to arrive at our predicted ratings.'

<img src="images/matrixfactorization.png" alt="matrix factorization image, showing the factor matrices" width=700>

[Image Source](https://medium.com/@connectwithghosh/simple-matrix-factorization-example-on-the-movielens-dataset-using-pyspark-9b7e3f567536)

A bit more on Matrix Factorization, [Google's Recommendations Systems crash course](https://developers.google.com/machine-learning/recommendation/collaborative/matrix)

#### Advantages:
- Personalized. You’re special!

#### Disadvantages:
- Can require a lot of computation, especially as these matrices get larger
- Cold start: need to have a lot of ratings to be worthwhile
- Popularity Bias: biased towards items that are popular. May not capture people’s unique tastes.

Matrix factorization methods include Singular Value Decomposition (SVD) and Alternating Least Squares (ALS)

I'll note that there are differences between _explicit_ and _implicit_ ratings.

- **_Explicit_** data is gathered from users when we ask a user to rate an item on some scale
    - Pros: concrete rating system, can assume users actually feel the way they input and thus can extrapolate from those preferences
    - Cons: not all users might input their preferences
- **_Implicit_** data is gathered from users without their direct input - a system logs the actions of a user
    - Pros: Easier to collect automatically, thus have more data from more users without those users needing to go through extra steps
    - Cons: More difficult to work with - how do we know what actions imply preference?

[Resource](https://www.cs.carleton.edu/cs_comps/0607/recommend/recommender/collaborativefiltering.html#:~:text=Implicit%20Data%20Collection,system%20has%20to%20collect%20data.&text=Explicit%20data%20gathering%20is%20easy,data%20to%20predict%20future%20ratings.)

## And now, in code!

### Reading in the data and simple EDA

#### Data Source:

https://www.kaggle.com/rounakbanik/the-movies-dataset

In [ ]:
# Import libraries, round 1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import Counter

In [ ]:
# load in data and check it out
df = pd.read_csv('ratings.csv') 
print(df.shape) 
df.head(10) 

### Ratings

In [ ]:
# check value_counts
ratings = df['rating'].value_counts()
ratings

In [ ]:
ratings_sorted = dict(zip(ratings.index, ratings))

In [ ]:
# plot distribution in matplotlib
plt.figure(figsize=(8,6))
plt.bar(ratings_sorted.keys(), ratings_sorted.values(), width=.4)
plt.xticks(np.arange(0, 5.1, step=0.5))
plt.xlabel("Rating")
plt.ylabel("# of Ratings")
plt.title("Distribution of Ratings")
plt.show()

### Users

In [ ]:
print("Number of users: ", df.userId.nunique()) 
print("Average Number of Reviews per User: ", df.shape[0]/df.userId.nunique())

In [ ]:
ratings_per_user = df['userId'].value_counts()
ratings_per_user = sorted(list(zip(ratings_per_user.index, ratings_per_user)))

plt.figure(figsize=(8,6))
plt.bar([r[0] for r in ratings_per_user], [r[1] for r in ratings_per_user])
plt.xlabel("User ID")
plt.ylabel("# of Reviews")
plt.title("Number of Reviews per User")
plt.show()

### Movies

In [ ]:
print("Number of movies: ", df.movieId.nunique())
print("Average Number of Reviews per Movie: ", df.shape[0]/df.movieId.nunique())

In [ ]:
# the movie IDs with the most ratings
df['movieId'].value_counts()[:10]

In [ ]:
ratings_per_movie = df['movieId'].value_counts()

plt.figure(figsize=(8, 6))
plt.hist(ratings_per_movie, bins=50)
plt.xlabel("# of Reviews")
plt.ylabel("# of Movies")
plt.title("Distribution of the Number of Ratings Per Movie")
plt.show()

## Fitting Recommendation System Models

One of the easiest libraries to use for recommendation systems is Surprise.

Surprise has some basic algorithms - like `BaselineOnly` ([documentation](https://surprise.readthedocs.io/en/stable/basic_algorithms.html#surprise.prediction_algorithms.baseline_only.BaselineOnly)), which predicts a baseline estimate for a given user an item.

In [ ]:
# Uncomment and run the line below if surprise is not installed:
# !pip install surprise -q

# Standard Surprise Imports
from surprise import Dataset, Reader

# Surprise Models
from surprise import BaselineOnly, SVD, KNNBasic

# Surprise Metrics
from surprise import accuracy

# Surprise Model Evaluation
from surprise.model_selection import cross_validate, train_test_split

### Task 1 - Create a Surprise Dataset

In the cell below, 
- Isolate the `userId`, `movieId`, and `rating`, columns. 
- Store a dataframe with those three columns in the variable `data`.

In [ ]:
# YOUR CODE HERE

In [ ]:
# for Surprise, we only need three columns from the dataset
data = df[['userId', 'movieId', 'rating']]
# Tell Surprise how to parse the data
reader = Reader(line_format='user item rating', sep=',')
# Create a surprise dataset
data = Dataset.load_from_df(data, reader=reader)

### Task 2 - Cross Validate a Model

In the cell below,

- Create an instance of a `BaselineOnly` model.
    - Set the variable name to `baseline`.
- Pass the baseline model into Surprise's `cross_validate` function.
    - Set verbose to `True`
    - Save the output to the variable `baseline_metrics`.

In [ ]:
# YOUR CODE HERE

In [ ]:
baseline = BaselineOnly()

baseline_metrics = cross_validate(baseline, data, verbose=True)

**Let's take a look at `baseline_metrics`.**

In [ ]:
baseline_metrics

### Task 3 - Calculate Metrics

Calculate the average **root mean squared error** (rmse) and **mean absolute error** (mae). for the baseline model.
- Store the averages in a tuple called `baseline_error` with the following format:
    - ```(rmse, mae)```

In [ ]:
# YOUR CODE HERE

In [ ]:
baseline_error = (baseline_metrics['test_rmse'].mean(), 
                  baseline_metrics['test_mae'].mean())

print('Baseline RMSE:', baseline_error[0])
print('Baseline MAE:', baseline_error[1])

### Task 4 - Fit a KNN Basic Model

In the cell below,

- Create an instance of the `KNNBasic` class
    - Set the instance to the variable `knn_basic`
- Cross validate the model
    - Set the output to `knn_basic_metrics`

In [ ]:
# YOUR CODE HERE

In [ ]:
knn_basic = KNNBasic()

knn_basic_metrics = cross_validate(knn_basic, data, verbose=True)

### Task 5 - Fit another Surprise KNN Model

- [Surprise KNN Documentation](https://surprise.readthedocs.io/en/stable/knn_inspired.html)

In [ ]:
# Your code here

### Task 6 - Create a train test split

In [ ]:
# YOUR CODE HERE

In [ ]:
trainset, testset = train_test_split(data, test_size=.2)

### Task 7 - Fit Singular Value Decomposition (SVD) Model.
> [Surprise SVD Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD)


In the cell below,

- Create an instance of the `SVD` class
    - Set the variable name to `svd`
- Fit the model to the training data

In [ ]:
# YOUR CODE HERE

In [ ]:
svd = SVD()
svd.fit(trainset)

### Task 8 - Calculate the test metrics

In the cell below,
- Generate predictions for the test data.
- Calculate RMSE and MAE
    - Store the error in a tuple called `svd_error` with the following format
        - `(rmse, mae)`


In [ ]:
# YOUR CODE HERE

In [ ]:
predictions = svd.test(testset)
rmse = accuracy.rmse(predictions, verbose=False)
mae = accuracy.mae(predictions, verbose=False)

svd_error = (rmse, mse)

print('SVD Test RMSE:', rmse)
print('SVD Test MAE: ', mae)

### Let's look at predictions for a second:

In [ ]:
predictions[:10]

In [ ]:
print("Number of users: ", df.userId.nunique()) 
print("Number of movies: ", df.movieId.nunique()) 

In [ ]:
user = 5
item = 141
svd.predict(user, item)

### Grid Search

In [ ]:
from surprise.model_selection import GridSearchCV

params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD ,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)